In [1]:
%matplotlib widget

import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy.linalg as la

from skimage import io
from matplotlib.patches import ConnectionPatch

# TeX typesetting
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

# Aesthetics
plt.style.use('seaborn-pastel')

In [2]:
def decomposeP(P):
    '''
        The input P is assumed to be a 3-by-4 homogeneous camera matrix.
        The function returns a homogeneous 3-by-3 calibration matrix K,
        a 3-by-3 rotation matrix R and a 3-by-1 vector c such that
        K*R*[eye(3), -c] = P.

    '''

    W = np.array([[0, 0, 1],
                  [0, 1, 0],
                  [1, 0, 0]])

    # calculate K and R up to sign
    Qt, Rt = np.linalg.qr((W.dot(P[:,0:3])).T)
    K = W.dot(Rt.T.dot(W))
    R = W.dot(Qt.T)

    # correct for negative focal length(s) if necessary
    D = np.array([[1, 0, 0],
                  [0, 1, 0],
                  [0, 0, 1]])
    if K[0,0] < 0:
        D[0,0] = -1
    if K[1,1] < 0:
        D[1,1] = -1
    if K[2,2] < 0:
        D[2,2] = -1
    K = K.dot(D)
    R = D.dot(R)

    # calculate c
    c = -R.T.dot(np.linalg.inv(K).dot(P[:,3]))

    return K, R, c

# Q1

## (a)

In [3]:
lego1 = io.imread("../assets/assignment-3/lego1.jpg")
lego2 = io.imread("../assets/assignment-3/lego2.jpg")

In [4]:
fig,ax = plt.subplots()
ax.imshow(lego1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
lego1_cam_corres = np.empty((28,3))

In [6]:
# X
lego1_cam_corres[0] = [1548.52,1840.55,1]
lego1_cam_corres[1] = [1615.52,1805.56,1]
lego1_cam_corres[2] = [1747.59,1734.53,1]
lego1_cam_corres[3] = [1874.48,1667.52,1]
lego1_cam_corres[4] = [1993.48,1604.53,1]
lego1_cam_corres[5] = [2112.45,1542.50,1]
lego1_cam_corres[6] = [2220.60,1483.51,1]
lego1_cam_corres[7] = [2325.48,1425.50,1]
lego1_cam_corres[8] = [2428.53,1367.50,1]

# Y
lego1_cam_corres[9] = [1306.50,1793.51,1]
lego1_cam_corres[10] = [1072.50,1747.50,1]
lego1_cam_corres[11] = [844.50,1701.51,1]
lego1_cam_corres[12] = [627.50,1657.51,1]
lego1_cam_corres[13] = [407.50,1614.5,1]
lego1_cam_corres[14] = [197.51,1570.50,1]

# Z
lego1_cam_corres[15] = [1552.50,1760.54,1]
lego1_cam_corres[16] = [1552.50,1679.50,1]
lego1_cam_corres[17] = [1552.50,1599.52,1]
lego1_cam_corres[18] = [1552.50,1517.54,1]
lego1_cam_corres[19] = [1552.50,1434.50,1]
lego1_cam_corres[20] = [1552.50,1357.50,1]
lego1_cam_corres[21] = [1552.50,1269.57,1]
lego1_cam_corres[22] = [1552.50,1187.51,1]
lego1_cam_corres[23] = [1552.50,1102.56,1]
lego1_cam_corres[24] = [1552.50,1019.51,1]
lego1_cam_corres[25] = [1552.50,937.483,1]
lego1_cam_corres[26] = [1552.50,851.521,1]
lego1_cam_corres[27] = [1552.50,766.51,1]

In [7]:
fig,ax = plt.subplots()
ax.imshow(lego1)
ax.plot(lego1_cam_corres[:, [0]], lego1_cam_corres[:, [1]], 'yo', markersize=4)

# origin and arrow heads
x0 = lego1_cam_corres[0][[0,1]]
x1 = lego1_cam_corres[8][[0,1]]
x2 = lego1_cam_corres[14][[0,1]]
x3 = lego1_cam_corres[27][[0,1]]

# arrow connections
coordsA = "data"
coordsB = "data"
conY = ConnectionPatch(x0, x1, coordsA, coordsB,
                      arrowstyle="-|>")
conX = ConnectionPatch(x0, x2, coordsA, coordsB,
                      arrowstyle="-|>")
conZ = ConnectionPatch(x0, x3, coordsA, coordsB,
                      arrowstyle="-|>")

# aesthetics.
conX.set_color([1.0,0,0])
conX.set_linewidth(2)
conY.set_color([1.0,0,0])
conY.set_linewidth(2)
conZ.set_color([1.0,0,0])
conZ.set_linewidth(2)

# add connections to axes
ax.add_artist(conX)
ax.add_artist(conY)
ax.add_artist(conZ)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# lego dimensions (mm)
h = 9.6
w = 32
b = 16

In [9]:
x = [[i*w + b,0,0,1] for i in range(0,8)]
y = [[0,i*w,0,1] for i in range(1,7)]
z = [[0,0,i*h,1] for i in range(1,14)]
world_corres = np.array([[0,0,0,1]] + x + y + z)

In [10]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot3D(world_corres[:,0],world_corres[:,1],world_corres[:,2], 'ro')

# axis labels
ax.set_xlabel(r'$X(mm)$')
ax.set_ylabel(r'$Y(mm)$')
ax.set_zlabel(r'$Z(mm)$')

# make the panes transparent
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

# make the grid lines transparent
ax.xaxis._axinfo["grid"]['color'] =  (242/255,242/255,242/255,1.0)
ax.yaxis._axinfo["grid"]['color'] =  (242/255,242/255,242/255,1.0)
ax.zaxis._axinfo["grid"]['color'] =  (242/255,242/255,242/255,1.0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## (c)

In [11]:
# small utils
flatten = lambda l: [item for sl in l for item in sl]
zeros = np.zeros(4)

In [12]:
# stack correspondences
lego1_A = []
for i in range(len(world_corres)):
    X = world_corres[i]
    xi, yi, _ = lego1_cam_corres[i]
    x = flatten([zeros,-X, yi*X])
    y = flatten([X, zeros, -xi*X])
    lego1_A.append(x)
    lego1_A.append(y)
lego1_A = np.array(lego1_A)

In [13]:
_, _, lego1_vh = la.svd(lego1_A)
lego1_P = lego1_vh[-1,:].reshape((3,4))
print(lego1_P)

[[ 2.21419526e-03 -2.88747997e-03 -1.41363984e-04  6.44022641e-01]
 [-4.25733657e-04 -2.81787236e-04 -3.65312618e-03  7.64988758e-01]
 [ 2.85448133e-07  1.91968592e-07 -9.73180520e-08  4.15370465e-04]]


In [14]:
lego1_K,lego1_R,lego1_c = decomposeP(lego1_P)

In [15]:
lego1_K / lego1_K[2,2]

array([[1.01588835e+04, 1.97985883e+02, 7.15862960e+02],
       [0.00000000e+00, 1.02214944e+04, 1.40759929e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [16]:
lego1_R

array([[ 0.55782386, -0.82995877, -0.00098891],
       [-0.22646355, -0.15106197, -0.96223414],
       [ 0.79846528,  0.53698111, -0.27222138]])

In [17]:
lego1_c

array([-969.26410858, -538.03139178,  363.86574805])

# Q2

## (a)

In [18]:
fig,ax = plt.subplots()
ax.imshow(lego2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
lego2_cam_corres = np.empty((28,3))

In [20]:
# X
lego2_cam_corres[0] = [989.50 , 1772.50, 1]
lego2_cam_corres[1] = [1093.50, 1754.50, 1]
lego2_cam_corres[2] = [1299.50, 1715.50, 1]
lego2_cam_corres[3] = [1499.50, 1676.50, 1]
lego2_cam_corres[4] = [1693.50, 1642.50, 1]
lego2_cam_corres[5] = [1882.50, 1608.50, 1]
lego2_cam_corres[6] = [2067.60, 1573.50, 1]
lego2_cam_corres[7] = [2246.50, 1538.50, 1]
lego2_cam_corres[8] = [2422.50, 1504.50, 1]


# Y
lego2_cam_corres[9]  = [853.50, 1713.54, 1]
lego2_cam_corres[10]  = [722.50, 1656.52, 1]
lego2_cam_corres[11] = [594.50, 1599.50, 1]
lego2_cam_corres[12] = [472.50, 1545.50, 1]
lego2_cam_corres[13] = [355.50, 1495.50, 1]
lego2_cam_corres[14] = [241.50, 1442.50, 1]

# Z
lego2_cam_corres[15] = [990.50, 1699.50, 1]
lego2_cam_corres[16] = [991.50, 1625.50, 1]
lego2_cam_corres[17] = [991.50, 1551.50, 1]
lego2_cam_corres[18] = [992.50, 1478.50, 1]
lego2_cam_corres[19] = [992.50, 1402.50, 1]
lego2_cam_corres[20] = [992.50, 1328.50, 1]
lego2_cam_corres[21] = [992.50, 1252.50, 1]
lego2_cam_corres[22] = [992.50, 1177.50, 1]
lego2_cam_corres[23] = [992.50, 1101.50, 1]
lego2_cam_corres[24] = [994.50, 1026.53, 1]
lego2_cam_corres[25] = [994.50, 951.50, 1]
lego2_cam_corres[26] = [995.50, 874.50, 1]
lego2_cam_corres[27] = [996.50, 789.50, 1]

In [30]:
fig,ax = plt.subplots()
ax.imshow(lego2)
ax.plot(lego2_cam_corres[:, [0]], lego2_cam_corres[:, [1]], 'yo', markersize=4)

# origin and arrow heads
x0 = lego2_cam_corres[0][[0,1]]
x1 = lego2_cam_corres[8][[0,1]]
x2 = lego2_cam_corres[14][[0,1]]
x3 = lego2_cam_corres[27][[0,1]]

coordsA = "data"
coordsB = "data"
conY = ConnectionPatch(x0, x1, coordsA, coordsB,
                      arrowstyle="-|>")
conX = ConnectionPatch(x0, x2, coordsA, coordsB,
                      arrowstyle="-|>")
conZ = ConnectionPatch(x0, x3, coordsA, coordsB,
                      arrowstyle="-|>")

# aesthetics
conX.set_color([1.0,0,0])
conX.set_linewidth(2)
conY.set_color([1.0,0,0])
conY.set_linewidth(2)
conZ.set_color([1.0,0,0])
conZ.set_linewidth(2)

# add connnections to axes
ax.add_artist(conX)
ax.add_artist(conY)
ax.add_artist(conZ)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
# stack correspondences
lego2_A = []
for i in range(len(world_corres)):
    X = world_corres[i]
    xi, yi, _ = lego2_cam_corres[i]
    x = flatten([zeros,-X, yi*X])
    y = flatten([X, zeros, -xi*X])
    lego2_A.append(x)
    lego2_A.append(y)
lego2_A = np.array(lego2_A)

In [38]:
_, _, lego2_vh = la.svd(lego2_A)
lego2_P = lego2_vh[-1,:].reshape((3,4))
print(lego2_P)

[[-3.44932524e-03  1.84988320e-03  9.52411705e-05 -4.87449000e-01]
 [ 2.30930612e-04  4.22745665e-04  3.95716210e-03 -8.73133444e-01]
 [-2.10502341e-07 -2.92004333e-07  1.18579858e-07 -4.92435491e-04]]


In [39]:
lego2_K,lego2_R,lego2_c = decomposeP(lego2_P)

In [40]:
lego2_K / lego2_K[2,2]

array([[ 1.02387926e+04, -3.08263102e+01,  1.37296197e+03],
       [ 0.00000000e+00,  1.03127170e+04,  2.06897030e+03],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [41]:
lego2_R

array([[-0.81390078,  0.58082162, -0.01455215],
       [ 0.17051429,  0.26273407,  0.94968189],
       [-0.55541911, -0.77046548,  0.31287785]])

In [42]:
lego2_c

array([ -688.36031714, -1039.1701425 ,   371.83252723])

## (b)

In [43]:
# distance (mm)
dist = la.norm(lego2_c - lego1_c)

# angle (radians)
M1 = lego1_K @ lego1_R
M2 = lego2_K @ lego2_R

p_axis1 = la.det(M1) * M1[2, :]
p_axis2 = la.det(M2) * M2[2, :]
ang = np.arccos(np.dot(p_axis1, p_axis2) / (la.norm(p_axis1) * la.norm(p_axis2)))

In [44]:
print('The cameras were {:.4f} mm apart and they have angle of {:.4f} '
         'between their respective axis'.format(dist, ang * 180.00/np.pi))

The cameras were 574.5524 mm apart and they have angle of 19.5448 between their respective axis


In [45]:
dx1 = M1[0, :] / la.norm(M1[0, :])
dy1 = M1[1, :] / la.norm(M1[1, :]) 
dz1 = p_axis1 / la.norm(p_axis1)

dx2 = M2[0, :] / la.norm(M2[0, :])
dy2 = M2[1, :] / la.norm(M2[1, :])
dz2 = p_axis2 / la.norm(p_axis2)

## (c)

In [46]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot3D(world_corres[:,0],world_corres[:,1],world_corres[:,2], 'o', color='orange', label='object')

# world axes labels
ax.set_xlabel(r'$X(mm)$')
ax.set_ylabel(r'$Y(mm)$')
ax.set_zlabel(r'$Z(mm)$')

# camera1 centre and camera1 axes directional axes
x1,y1,z1 = lego1_c
x2,y2,z2 =  lego1_R.T @ [100, 0, 0] + lego1_c
x3,y3,z3 =  lego1_R.T @ [0, 100, 0] + lego1_c
x4,y4,z4 =  lego1_R.T @ [0, 0, 100] + lego1_c

ax.scatter3D(x1, y1, z1, color="pink", label='camera 1')
ax.plot3D([x1, x2], [y1,y2], [z1,z2], color='pink')
ax.plot3D([x1, x3], [y1,y3], [z1,z3], color='pink')
ax.plot3D([x1, x4], [y1,y4], [z1,z4], color='pink')


# camera2 centre and camera2 axes directional axes
x5,y5,z5 = lego2_c
x6,y6,z6 = lego2_R.T @ [-100, 0, 0] + lego2_c
x7,y7,z7 = lego2_R.T @ [0, -100, 0] + lego2_c
x8,y8,z8 = lego2_R.T @ [0, 0, -100] + lego2_c

ax.scatter3D(x5, y5, z5, color="magenta", label='camera 2')
ax.plot3D([x5, x6], [y5,y6], [z5,z6], color='magenta')
ax.plot3D([x5, x7], [y5,y7], [z5,z7], color='magenta')
ax.plot3D([x5, x8], [y5,y8], [z5,z8], color='magenta')


# labels for camera-1 axes
ax.text(x2, y2 - 50, z2, r'$x_1$', size=10, zorder=1, color='k')
ax.text(x3, y3, z3 - 50, r'$y_1$', size=10, zorder=1, color='k')
ax.text(x4 + 50, y4, z4, r'$z_1$', size=10, zorder=1, color='k')

# lables for camera2 axes
ax.text(x6, y6 - 50, z6, r'$x_2$', size=10, zorder=1, color='k')
ax.text(x7, y7, z7 - 50, r'$y_2$', size=10, zorder=1, color='k')
ax.text(x8 + 50, y8, z8, r'$z_2$', size=10, zorder=1, color='k')

# make the panes transparent
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))

# make the grid lines transparent
ax.xaxis._axinfo["grid"]['color'] =  (242/255,242/255,242/255,1.0)
ax.yaxis._axinfo["grid"]['color'] =  (242/255,242/255,242/255,1.0)
ax.zaxis._axinfo["grid"]['color'] =  (242/255,242/255,242/255,1.0)

ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Q3

In [47]:
# columns of P
lego1_p1 = lego1_P[:, 0]
lego1_p2 = lego1_P[:, 1]
lego1_p3 = lego1_P[:, 2]
lego1_p4 = lego1_P[:, 3]

In [48]:
# Homogenize vectors
lego1_p1 = lego1_p1 / lego1_p1[2]
lego1_p2 = lego1_p2 / lego1_p2[2]
lego1_p3 = lego1_p3 / lego1_p3[2]
lego1_p4 = lego1_p4 / lego1_p4[2]

In [49]:
# make an interval for axes lines
lego1_X1, lego1_Y1 = np.linspace(lego1_p4[0], lego1_p1[0], 500), np.linspace(lego1_p4[1], lego1_p1[1], 500)
lego1_X2, lego1_Y2 = np.linspace(lego1_p4[0], lego1_p2[0], 500), np.linspace(lego1_p4[1], lego1_p2[1], 500)
lego1_X3, lego1_Y3 = np.linspace(lego1_p4[0], lego1_p3[0], 500), np.linspace(lego1_p4[1], lego1_p4[1] - lego1_p3[1], 500)

In [50]:
# cap axes lines at some point because they will end up failling outside the image range.

fig,ax = plt.subplots()
ax.imshow(lego1)

ax.plot(lego1_X1[0:50], lego1_Y1[0:50], color='r')
ax.plot(lego1_X2[0:30], lego1_Y2[0:30], color='r')
ax.plot(lego1_X3[0:10], lego1_Y3[0:10], color='r')

ax.text(lego1_X1[51], lego1_Y1[51], r'X', size=10, zorder=1, color='k')
ax.text(lego1_X2[31], lego1_Y2[31], r'Y', size=10, zorder=1, color='k')
ax.text(lego1_X3[10], lego1_Y3[10], r'Z', size=10, zorder=1, color='k')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(1548.5161727109528, 1089.4375764569356, 'Z')

In [51]:
# columns of P'
lego2_p1 = lego2_P[:, 0]
lego2_p2 = lego2_P[:, 1]
lego2_p3 = lego2_P[:, 2]
lego2_p4 = lego2_P[:, 3]

In [52]:
# De-homogenize vectors
lego2_p1 = lego2_p1 / lego2_p1[2]
lego2_p2 = lego2_p2 / lego2_p2[2]
lego2_p3 = lego2_p3 / lego2_p3[2]
lego2_p4 = lego2_p4 / lego2_p4[2]

In [53]:
# make interval for axes lines
lego2_X1, lego2_Y1 = np.linspace(lego2_p4[0], lego2_p1[0], 500), np.linspace(lego2_p4[1], lego2_p1[1], 500)
lego2_X2, lego2_Y2 = np.linspace(lego2_p4[0], lego2_p2[0], 500), np.linspace(lego2_p4[1], lego2_p2[1], 500)
lego2_X3, lego2_Y3 = np.linspace(lego2_p4[0], lego2_p3[0], 500), np.linspace(lego2_p4[1], lego2_p4[1] - lego2_p3[1], 500)

In [54]:
# cap axes lines interval!
fig,ax = plt.subplots()

ax.imshow(lego2)
ax.plot(lego2_X1[0:30], lego2_Y1[0:30], color='r')
ax.plot(lego2_X2[0:30], lego2_Y2[0:30], color='r')
ax.plot(lego2_X3[0:10], lego2_Y3[0:10], color='r')

ax.text(lego2_X1[31], lego2_Y1[31], 'X', size=10, zorder=1, color='k')
ax.text(lego2_X2[31], lego2_Y2[31], 'Y', size=10, zorder=1, color='k')
ax.text(lego2_X3[10], lego2_Y3[10], 'Z', size=10, zorder=1, color='k')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(986.1324925500735, 1104.3288233073192, 'Z')

# Q4

### (a)

In [55]:
# Epipoles
e       = lego1_P @ np.append(lego2_c, 1.0)
e_prime = lego2_P @ np.append(lego1_c, 1.0)

In [56]:
homo_e = e / e[2]
homo_e

array([-1.23126444e+05,  4.45338262e+02,  1.00000000e+00])

In [57]:
homo_e_prime = e_prime / e_prime[2]
homo_e_prime

array([-2.15002904e+04, -1.30982534e+03,  1.00000000e+00])

## (b)

In [58]:
P_plus = lego1_P.T @ la.inv(lego1_P @ lego1_P.T)
cross_mat = np.array([[0, -e_prime[2], e_prime[1]], [e_prime[2], 0, -e_prime[0]], [-e_prime[1], e_prime[0], 0]])
F = cross_mat @ lego2_P @ P_plus
F

array([[-1.51637630e-06, -9.53310406e-05, -1.44251461e-01],
       [ 1.88362528e-05,  1.31954329e-06,  2.31865318e+00],
       [-7.93032954e-03, -2.04791668e+00, -6.44176206e+01]])

In [59]:
# matches
lego1_kp = [[2231.09, 1123.55, 1.0], [848.828, 750.32,  1.0], [197.712, 1422.63, 1.0], [2456.44, 459.50, 1.0], [1308.64, 1715.55, 1.0]] 
lego2_kp = [[2075.45, 1229.97, 1.0], [596.282, 753.286, 1.0], [242.334, 1314.18, 1.0], [2447.76, 626.87, 1.0], [854.66, 1641.60, 1.0]]

In [60]:
fig,ax = plt.subplots(1,2)
ax[0].set_axis_off()
ax[1].set_axis_off()
ax[0].imshow(lego1)
ax[1].imshow(lego2)

xp = np.array([0, np.shape(lego1)[1] - 5])
x  = np.array([0, np.shape(lego2)[1] - 5])
for i in range(5):
    l   = F.T @ np.array(lego2_kp[i])
    lp  = F @ np.array(lego1_kp[i])
    l_norm = l / la.norm(l)
    lp_norm = lp / la.norm(lp)
    ap,bp,cp = lp_norm
    a,b,c = l_norm
    yp = -(xp*ap + cp) / bp
    y  = -(x*a + c) / b
    
    ax[0].plot(x, y)
    ax[0].scatter(lego1_kp[i][0], lego1_kp[i][1], s=10, zorder=99, edgecolor='k')
    
    ax[1].plot(xp, yp)
    ax[1].scatter(lego2_kp[i][0], lego2_kp[i][1], s=10, zorder=99, edgecolor='k')
    
plt.subplots_adjust(wspace=0, hspace=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …